<h1>🚀 蒼狼 AI v2.2 - 一鍵啟動器</h1>

歡迎使用！點擊下方儲存格左側的 ▶️ **執行按鈕**來啟動您的 AI 分析平台。

**啟動流程：**
1. **環境設定**：自動下載專案程式碼並安裝所有必要的 Python 和 Node.js 套件。
2. **啟動服務**：同時啟動後端 FastAPI 和前端 Next.js 服務。
3. **建立通道**：使用 `ngrok` 建立一個安全的公開網址，以便您存取服務。
4. **嵌入介面**：**您的應用程式使用者介面將會直接顯示在此儲存格的下方**，無需離開 Colab。

---
**提示**：首次執行可能需要幾分鐘時間來下載和安裝套件，請耐心等候。

In [ ]:
#@title ▶️ 點此一鍵啟動 Wolf AI 平台
import os
import subprocess
import threading
from pyngrok import ngrok
from IPython.display import display, IFrame, HTML
import time
import requests

# --- 設定 --- 
REPO_URL = "https://github.com/hsp1234-web/wolfai_v2_jules.git"
REPO_PATH = "/content/wolfai_v2_jules"
NGROK_AUTH_TOKEN = "" # 建議填寫您的 ngrok Authtoken 以獲得更穩定連線

def stream_output(process, log_file):
    """將子進程的輸出即時寫入日誌檔案"""
    with open(log_file, 'w') as f:
        for line in iter(process.stdout.readline, ''):
            f.write(line)
            f.flush()

print("⏳ [1/5] 開始準備環境...")

if os.path.exists(REPO_PATH):
    print(f"   -> 倉庫目錄 {REPO_PATH} 已存在，正在更新...")
    subprocess.run(f'cd {REPO_PATH} && git pull', shell=True, check=True)
else:
    print(f"   -> 正在從 {REPO_URL} 下載專案...")
    subprocess.run(f'git clone {REPO_URL} {REPO_PATH}', shell=True, check=True)

print("✅ [2/5] 專案程式碼已備妥！")
print("⏳ [3/5] 正在安裝後端與前端依賴套件 (此步驟可能需要數分鐘)...")

# 安裝所有依賴
subprocess.run(f'pip install -q -r {REPO_PATH}/backend/requirements.txt', shell=True, check=True)
subprocess.run(f'npm install --prefix {REPO_PATH}/frontend', shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) # 隱藏 npm 的大量輸出

print("✅ [4/5] 所有依賴已安裝！")
print("⏳ [5/5] 正在啟動後端與前端服務...")

# 啟動後端
backend_process = subprocess.Popen(['uvicorn', 'main:app', '--host', '0.0.0.0', '--port', '8000'], cwd=f'{REPO_PATH}/backend', stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
threading.Thread(target=stream_output, args=(backend_process, 'backend.log')).start()
print("   -> 後端 FastAPI 服務已在背景啟動。")

print("   -> Checking backend health (/api/v1/health)...")
start_time = time.time()
backend_healthy = False
while time.time() - start_time < 60:
    try:
        response = requests.get("http://localhost:8000/api/v1/health")
        if response.status_code == 200:
            print("   -> Backend is healthy!")
            backend_healthy = True
            break
    except requests.exceptions.ConnectionError:
        # Backend not up yet
        pass
    except requests.exceptions.RequestException as e:
        print(f"   -> An error occurred during health check: {e}")
    time.sleep(2)

if not backend_healthy:
    print("   -> ERROR: Backend health check timed out after 60 seconds.")
    # Log backend output for debugging
    print("   -> Displaying backend logs for diagnostics...")
    with open('backend.log', 'r') as f:
        backend_logs = f.read()
        if backend_logs:
            print("--- Backend Log ---")
            print(backend_logs)
            print("--- End Backend Log ---")
        else:
            print("   -> Backend log is empty.")
    raise RuntimeError("Backend service failed to start or become healthy within 60 seconds.")

# 啟動前端
frontend_process = subprocess.Popen(['npm', 'run', 'dev'], cwd=f'{REPO_PATH}/frontend', stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
threading.Thread(target=stream_output, args=(frontend_process, 'frontend.log')).start()
print("   -> 前端 Next.js 服務已在背景啟動。")

print("   -> Waiting for frontend service to become ready (checking frontend.log)...")
start_time = time.time()
frontend_ready = False
ready_messages = ["event - compiled client and server successfully", "Ready", "started server on 0.0.0.0:3000"]
while time.time() - start_time < 120:
    try:
        with open('frontend.log', 'r') as f:
            log_content = f.read()
            if any(msg in log_content for msg in ready_messages):
                print("   -> Frontend service is ready!")
                frontend_ready = True
                break
    except FileNotFoundError:
        # Log file might not be created yet
        pass
    except Exception as e:
        print(f"   -> An error occurred while checking frontend log: {e}")
    time.sleep(2)

if not frontend_ready:
    print("   -> ERROR: Frontend readiness check timed out after 120 seconds.")
    print("   -> Displaying frontend logs for diagnostics...")
    try:
        with open('frontend.log', 'r') as f:
            frontend_logs = f.read()
            if frontend_logs:
                print("--- Frontend Log ---")
                print(frontend_logs)
                print("--- End Frontend Log ---")
            else:
                print("   -> Frontend log is empty or could not be read.")
    except FileNotFoundError:
        print("   -> Frontend log file (frontend.log) not found.")
    raise RuntimeError("Frontend service failed to become ready within 120 seconds.")

# 設定 ngrok
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("   -> All services ready. Establishing ngrok tunnel...") # New print statement

# 建立 ngrok 通道並顯示
try:
    public_url = ngrok.connect(3000).public_url # Assuming frontend runs on port 3000
    print(f"
🎉🎉🎉 您的 AI 平台已上線！ 🎉🎉🎉")
    print(f"🌐 公開網址: {public_url}")
    display(HTML("<h2>↓ 您可以直接在下方操作 WolfAI 介面 ↓</h2>"))
    display(IFrame(public_url, width='100%', height=800))
except Exception as e:
    print(f"
🛑 錯誤：無法建立 ngrok 通道。請檢查您的 Authtoken 或 Colab 網路狀態。錯誤訊息: {e}")